In [13]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.draw import polygon
from scipy.ndimage import distance_transform_edt
import pylbm

# Function to create a polygon and generate its SDF
def create_polygon(sides, radius, image_size):
    angles = np.linspace(0, 2 * np.pi, sides, endpoint=False)
    x = radius * np.cos(angles) + image_size // 2
    y = radius * np.sin(angles) + image_size // 2
    rr, cc = polygon(y, x)
    
    image = np.zeros((image_size, image_size), dtype=np.uint8)
    image[rr, cc] = 1
    
    return image

# Function to compute the SDF of the polygon
def compute_sdf(image):
    return distance_transform_edt(image) - distance_transform_edt(1 - image)

# Function to generate LBM velocity field using pylbm
def compute_velocity_field(sdf):
    image_size = sdf.shape[0]
    dx = 1.0
    la = 1.0
    rho0 = 1.0
    mu = 0.01
    Tf = 1000.0

    # Define the geometry
    geom = pylbm.Geometry(
        {
            'box': {
                'x': [0, image_size],
                'y': [0, image_size],
                'label': 0
            },
            'elements': [
                {
                    'type': 'rectangle',
                    'bounds': [[0, 0], [image_size, image_size]],
                    'label': 1
                }
            ]
        }
    )
    
    # Create the scheme
    d = {
        'box': {
            'x': [0, image_size],
            'y': [0, image_size],
            'label': [0, 1, 2, 3]
        },
        'elements': [geom],
        'space_step': dx,
        'schemes': [
            {
                'velocities': list(range(9)),
                'conserved_moments': ['rho', 'q'],
                'polynomials': [
                    1,
                    'ux',
                    'uy',
                    'ux**2+uy**2',
                    'ux*uy',
                    'ux**2-uy**2',
                    'ux**2+uy**2',
                    'ux**2*uy**2',
                    'ux**2*uy**2',
                ],
                'relaxation_parameters': [0., 1.9, 1.9, 1.0, 1.2, 1.2, 1.0, 1.2, 1.2],
                'equilibrium': [
                    'rho',
                    'q',
                    'ux',
                    'uy',
                    'ux**2+uy**2',
                    'ux*uy',
                    'ux**2-uy**2',
                    'ux**2+uy**2',
                    'ux**2*uy**2',
                ],
            }
        ],
        'init': {
            'rho': rho0,
            'q': [0., 0.],
        },
        'parameters': {'LA': la},
        'boundary_conditions': {
            0: {'method': pylbm.bc.BounceBack},
        },
        'generator': pylbm.generator.CythonGenerator,
    }

    # Initialize the simulation
    simu = pylbm.Simulation(d)
    
    # Set obstacle cells based on sdf
    for i in range(image_size):
        for j in range(image_size):
            if sdf[i, j] < 0:
                simu.m[0][i, j] = 0.0

    simu.apply_bc()
    while simu.t < Tf:
        simu.one_time_step()

    ux = simu.m[1]
    uy = simu.m[2]
    velocity_field = np.stack((ux, uy), axis=-1)

    return velocity_field

# Parameters
image_size = 256
radius = 50
sides_list = [3, 4, 5, 6, 12]

# Generate dataset
num_samples_per_shape = 20  # Reduced for testing purposes
train_data = []

for sides in sides_list:
    for _ in range(num_samples_per_shape):
        # Randomize position and orientation
        image = create_polygon(sides, radius, image_size)
        sdf = compute_sdf(image)
        velocity_field = compute_velocity_field(sdf)
        
        train_data.append((sdf, velocity_field))

# Convert to numpy arrays for storage
sdf_data = np.array([data[0] for data in train_data])
velocity_data = np.array([data[1] for data in train_data])

# Save dataset to disk
np.save('sdf_data.npy', sdf_data)
np.save('velocity_data.npy', velocity_data)


[0] ERROR    pylbm.validator in function validate line 357
Check the dictionary for Geometry class
[0] ERROR    pylbm.validator in function validate line 358
   | box: 
   |     x: [0, 256] 
   |     y: [0, 256] 
   |     label: 0 
>>>| elements: 
>>>|     0:
must be of element type
   | inittype: moments 



SystemExit: 